In [10]:
import csv
import sqlite3
import pandas as pd

In [11]:
data_df = pd.read_csv('ABCLicenseData.csv', header=0, nrows=100, index_col=0, na_values=['n/a'])

In [12]:
data_df.to_sql('ABC_data_table', sqlite3.connect("ABC.db"), if_exists = "replace")

In [14]:
con = sqlite3.connect("ABC.db")
data_df = pd.read_sql_query("SELECT * FROM ABC_data_table", con)
data_df.head()

,CodesAndRegsLicenseDataID,LicenseID,Description,LicenseName,Location,StreetNumber,Prefix,StreetName,Suffix,Longitude,...,DISTRICT,NEIGHBORHOOD,ZONING,EndorsementType,EndorsementTypeDescription,EndorsementStatusDescription,EndorsementIssuedDate,AGENCY,GPSX,GPSY
0,1,5677,Alcoholic Beverage Annual,TUMBLEWEED SOUTHWEST GRILL,4255 OUTER LOOP,4255,,OUTER,LOOP,-85.682816,...,24,None,C1,RLD,NQ2 Retail Drink (Restaurant),ACTIVE,None,Codes and Regulations,1228336,235821
1,2,5677,Alcoholic Beverage Annual,TUMBLEWEED SOUTHWEST GRILL,4255 OUTER LOOP,4255,,OUTER,LOOP,-85.682816,...,24,None,C1,SD,Sunday Drink,ACTIVE,None,Codes and Regulations,1228336,235821
2,3,5679,Alcoholic Beverage Annual,TUMBLEWEED SOUTHWEST GRILL,3550 SPRINGHURST COMMONS DR,3550,,SPRINGHURST COMMONS,DR,-85.561421,...,17,None,C1,RLD,NQ2 Retail Drink (Restaurant),ACTIVE,None,Codes and Regulations,1264070,293100
3,4,5679,Alcoholic Beverage Annual,TUMBLEWEED SOUTHWEST GRILL,3550 SPRINGHURST COMMONS DR,3550,,SPRINGHURST COMMONS,DR,-85.561421,...,17,None,C1,SD,Sunday Drink,ACTIVE,None,Codes and Regulations,1264070,293100
4,5,4642,Alcoholic Beverage Annual,CARDINAL HALL OF FAME CAFE,2745 CRITTENDEN DR,2745,,CRITTENDEN,DR,-85.748965,...,15,FAIRGROUNDS,M2,SBL,Supplemental Bar,ACTIVE,None,Codes and Regulations,1209698,259779
